<a href="https://colab.research.google.com/github/palarunava/machine-learning-courses/blob/main/machine-learning-specialization/supervised-ml-regression-classification/C1_W2_Gradient_Descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Get the data**

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yasserh/housing-prices-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'housing-prices-dataset' dataset.
Path to dataset files: /kaggle/input/housing-prices-dataset


In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv(path + "/Housing.csv")

In [4]:
data['mainroad'] = data['mainroad'].map({'yes': 1, 'no': 0})
data['guestroom'] = data['guestroom'].map({'yes': 1, 'no': 0})
data['basement'] = data['basement'].map({'yes': 1, 'no': 0})
data['hotwaterheating'] = data['hotwaterheating'].map({'yes': 1, 'no': 0})
data['airconditioning'] = data['airconditioning'].map({'yes': 1, 'no': 0})
data['prefarea'] = data['prefarea'].map({'yes': 1, 'no': 0})
data['furnishingstatus'] = data['furnishingstatus'].map({'furnished': 2, 'semi-furnished': 1, 'unfurnished': 0})

In [ ]:
data

In [19]:
import math
num_samples = data.shape[0]
train_valid_test_split_ratio = [75, 20, 5] # 75:20:5 split
train_num_samples = math.floor((num_samples / 100) * train_valid_test_split_ratio[0])
valid_num_samples = math.floor((num_samples / 100) * train_valid_test_split_ratio[1])
test_num_samples = num_samples - train_num_samples - valid_num_samples
print(f"Number of training samples: {train_num_samples}")
print(f"Number of validation samples: {valid_num_samples}")
print(f"Number of testing samples: {test_num_samples}")

Number of training samples: 408
Number of validation samples: 109
Number of testing samples: 28


In [21]:
data_train = data.iloc[:train_num_samples, :]
data_valid = data.iloc[train_num_samples:(train_num_samples + valid_num_samples), :]
data_test = data.iloc[(train_num_samples + valid_num_samples):, :]

In [6]:
X_train = data.iloc[:, 1:]
Y_train = data.iloc[:, 0]
assert X_train.shape[0] == Y_train.shape[0]
print("Information about the data:")
print(f"Number of training samples: {X_train.shape[0]}")
print(f"Number of features: {X_train.shape[1]}")
print("\n")
X_train.info()

Information about the data:
Number of training samples: 545
Number of features: 12


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   area              545 non-null    int64
 1   bedrooms          545 non-null    int64
 2   bathrooms         545 non-null    int64
 3   stories           545 non-null    int64
 4   mainroad          545 non-null    int64
 5   guestroom         545 non-null    int64
 6   basement          545 non-null    int64
 7   hotwaterheating   545 non-null    int64
 8   airconditioning   545 non-null    int64
 9   parking           545 non-null    int64
 10  prefarea          545 non-null    int64
 11  furnishingstatus  545 non-null    int64
dtypes: int64(12)
memory usage: 51.2 KB


Normalize the data - Z score

In [7]:
X_norm = (X_train - X_train.mean()) / X_train.std()
#X_norm.to_numpy()
X_norm

,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,1.045766,1.402131,1.420507,1.376952,0.405251,-0.464888,-0.733865,-0.219063,1.471267,1.516299,1.803284,1.404995
1,1.755397,1.402131,5.400847,2.529700,0.405251,-0.464888,-0.733865,-0.219063,1.471267,2.676950,-0.553526,1.404995
2,2.216196,0.047235,1.420507,0.224204,0.405251,-0.464888,1.360148,-0.219063,-0.678439,1.516299,1.803284,0.091578
3,1.082630,1.402131,1.420507,0.224204,0.405251,-0.464888,1.360148,-0.219063,1.471267,2.676950,1.803284,1.404995
4,1.045766,1.402131,-0.569663,0.224204,0.405251,2.147110,1.360148,-0.219063,1.471267,1.516299,-0.553526,1.404995
...,...,...,...,...,...,...,...,...,...,...,...,...
540,-0.990968,-1.307661,-0.569663,-0.928544,0.405251,-0.464888,1.360148,-0.219063,-0.678439,1.516299,-0.553526,-1.221840
541,-1.267448,0.047235,-0.569663,-0.928544,-2.463081,-0.464888,-0.733865,-0.219063,-0.678439,-0.805002,-0.553526,0.091578
542,-0.705273,-1.307661,-0.569663,-0.928544,0.405251,-0.464888,-0.733865,-0.219063,-0.678439,-0.805002,-0.553526,-1.221840
543,-1.032440,0.047235,-0.569663,-0.928544,-2.463081,-0.464888,-0.733865,-0.219063,-0.678439,-0.805002,-0.553526,1.404995


In [140]:
Y_norm = (Y_train - Y_train.mean()) / Y_train.std()
Y_norm

,price
0,4.562174
1,4.000809
2,4.000809
3,3.982096
4,3.551716
...,...
540,-1.575421
541,-1.603676
542,-1.612845
543,-1.612845


In [115]:
w = np.zeros(X_norm.shape[1])
b = 0.
print(f"Initial weights: {w}")
print(f"Initial bias: {b}")

Initial weights: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Initial bias: 0.0


In [107]:
Y_predict = X_norm.dot(w) + b
Y_error = Y_predict - Y_norm
cost = 1/(2*X_norm.shape[0]) * (Y_error ** 2).sum()
dj_dw = Y_error.dot(X_norm) / X_norm.shape[0]
dj_db = np.sum(Y_error) / X_norm.shape[0]
w = w - 0.1 * dj_dw
b = b - 0.1 * dj_db
print(f'Interation: {iter + 1} Cost: {cost}')

Interation: 10000 Cost: 13106916364659.268


In [162]:
w = np.zeros(X_norm.shape[1])
b = 0
#w = np.random.random(X_norm.shape[1])
#b = np.random.random()
print(f"Initial weights: {w}")
print(f"Initial bias: {b}")
iterations = 2000
learning_rate = 0.03
for iter in range(iterations):
  Y_predict = X_norm.dot(w) + b
  Y_error = Y_predict - Y_norm
  cost = (Y_error ** 2).sum() / (2*X_norm.shape[0])
  dj_dw = Y_error.dot(X_norm) / X_norm.shape[0]
  dj_db = np.sum(Y_error) / X_norm.shape[0]
  w -= learning_rate * dj_dw
  b -= learning_rate * dj_db
  if (iter + 1) % 10 == 0:
    print(f'Interation: {iter + 1} Cost: {cost}')
  #print(f'dj_dw: {dj_dw}')
  #print(f'dj_db: {dj_db}')

print(f'w: {w}')
print(f'b: {b}')

Initial weights: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Initial bias: 0
Interation: 10 Cost: 0.25245509489422197
Interation: 20 Cost: 0.1851101110234203
Interation: 30 Cost: 0.16884990717573184
Interation: 40 Cost: 0.16417155190352528
Interation: 50 Cost: 0.16237639259761952
Interation: 60 Cost: 0.16145861954092652
Interation: 70 Cost: 0.16089951303304437
Interation: 80 Cost: 0.1605309276371112
Interation: 90 Cost: 0.16027969333897663
Interation: 100 Cost: 0.16010561487743502
Interation: 110 Cost: 0.15998372157015792
Interation: 120 Cost: 0.1598976467416952
Interation: 130 Cost: 0.15983640212553002
Interation: 140 Cost: 0.15979251252761778
Interation: 150 Cost: 0.1597608449901379
Interation: 160 Cost: 0.15973784684223324
Interation: 170 Cost: 0.15972104099216503
Interation: 180 Cost: 0.1597086879794465
Interation: 190 Cost: 0.15969955781811054
Interation: 200 Cost: 0.15969277484725558
Interation: 210 Cost: 0.15968771150812883
Interation: 220 Cost: 0.159683915135905
Interation: 230 Cost:

In [66]:
print(type(Y_error))
print(X_norm.shape)
dj_dw = Y_error.dot(X_norm)/X_norm.shape[0]
print(dj_dw.shape)
print(dj_dw.to_numpy())

<class 'pandas.core.series.Series'>
(545, 12)
(12,)
[-1000711.12691782  -684247.13747121  -966261.09057547  -785473.19197208
  -554311.7407424   -477052.72642098  -349236.09304122  -173767.70847778
  -845668.72516859  -717665.86968431  -615696.26380535  -568917.28955954]


In [40]:
def compute_gradient(X, y, w, b):
    #print(f'{X}\n')
    #print(f'{y}\n')
    #print(f'{w}\n')
    #print(f'{b}\n')
    """
    Computes the gradient for linear regression
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters
      b (scalar)       : model parameter

    Returns:
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w.
      dj_db (scalar):       The gradient of the cost w.r.t. the parameter b.
    """
    m,n = X.shape           #(number of examples, number of features)
    dj_dw = np.zeros((n,))
    #print(f'{dj_dw}\n')
    dj_db = 0.

    for i in range(m):
        err = (np.dot(X[i], w) + b) - y[i]
        for j in range(n):
            #print(f'Gradient')
            #print(f'{dj_dw[j]}\n')
            #print(f'{err}\n')
            #print(f'{X[i, j]}\n')
            dj_dw[j] = dj_dw[j] + err * X[i, j]
        dj_db = dj_db + err
    dj_dw = dj_dw / m
    dj_db = dj_db / m

    return dj_db, dj_dw

In [41]:
print(X_norm.shape)
print(Y_train.shape)
print(w_init.shape)
print(b_init)
dj_db, dj_dw = compute_gradient(X_norm.to_numpy(), Y_train.to_numpy(), w_init, b_init)
print(f'dj_db: {dj_db}')
print(f'dj_dw: {dj_dw}')

(545, 12)
(545,)
(12,)
0.0
dj_db: -4766729.247706422
dj_dw: [-1000711.12691782  -684247.1374712   -966261.09057547  -785473.19197208
  -554311.7407424   -477052.72642098  -349236.09304122  -173767.70847778
  -845668.72516859  -717665.86968431  -615696.26380535  -568917.28955954]
